<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Aprior算法" data-toc-modified-id="Aprior算法-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Aprior算法</a></span><ul class="toc-item"><li><span><a href="#Aprior-算法" data-toc-modified-id="Aprior-算法-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Aprior 算法</a></span></li><li><span><a href="#抽取关联规则" data-toc-modified-id="抽取关联规则-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>抽取关联规则</a></span></li><li><span><a href="#评估" data-toc-modified-id="评估-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>评估</a></span></li></ul></li><li><span><a href="#总结" data-toc-modified-id="总结-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>总结</a></span></li></ul></div>

亲和性分析, 通常用于找出在什么情况下两个对象经常一起出现.  
也叫, 购物篮分析(因为被用于找哪些商品经常一起出现)
应用场景:
 - 欺诈检测
 - 顾客区分
 - 软件优化
 - 产品推荐

# Aprior算法
Aprior 算法是典型和基础的亲和性分析算法, 从数据集中频繁出现的商品中选取共同出现的商品组成频繁项集,避免规则的指数级增长的搜索复杂度.  
Aprior首先确保规则在数据集中有足够的支持度(出现次数);  
其次,频繁项集的自己也是频繁项集.  
其他算法, 有Eclat和频繁项集挖掘算法(FP-growth)

数据集  
grouplens.org/datasets/movielens  
1million


In [1]:
import os
import pandas as pd
rating_filename = os.path.join(os.getcwd(), 'dataset', 'ml-1m', 'ratings.dat')
all_ratings = pd.read_csv(rating_filename, delimiter='::', header=None, names=["UserID", 'MovieID','Rating','Datetime'])
all_ratings["Datetime"] = pd.to_datetime(all_ratings["Datetime"], unit='s')
all_ratings[:5]

C:\Users\shelgor\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  after removing the cwd from sys.path.


,UserID,MovieID,Rating,Datetime
0,1,1193,5,2000-12-31 22:12:40
1,1,661,3,2000-12-31 22:35:09
2,1,914,3,2000-12-31 22:32:48
3,1,3408,4,2000-12-31 22:04:35
4,1,2355,5,2001-01-06 23:38:11


以上表示, 称为 稀疏矩阵  
实际上, 当矩阵60%以上为0时, 采用这种关系列表是更好的选择.  
稀疏矩阵, 看起来并不像矩阵.  

In [2]:
# 生成规则: 如果用户喜欢某些电影, 那么他们也会喜欢这部电影 
# 新建favorable列表, 若喜欢该电影则为True
all_ratings["Favorable"] = (all_ratings["Rating"]>3)
# 先选取部分数据集进行训练, 减少搜索空间, 提升Aprior算法速度
# 这里选取前200名用户进行打分
ratings = all_ratings[all_ratings["UserID"].isin(range(200))]
favorable_ratings = ratings[ratings["Favorable"]==True]

In [3]:
favorable_review_by_users = dict((k, frozenset(v.values))
                                for k,v in favorable_ratings.groupby("UserID")["MovieID"])
num_favorable_by_movie = ratings[["MovieID", "Favorable"]].groupby("MovieID").sum()
num_favorable_by_movie.sort_values(by="Favorable",ascending=False)[:5]

,Favorable
MovieID,
2858,106.0
2028,85.0
1196,83.0
260,80.0
3578,77.0


## Aprior 算法  
1. 初始化: 按照支持度要求, 生成质保函自己的项集;  
2. 查找现有频繁项集的超集, 生成备选集;  
3. 测试备选集是否满足频繁程度要求,舍弃不频繁的项集;
4. 如果未发现新的频繁项集, 步骤5输出; 否则, 以新的频繁项集返回步骤2;
5. 输出当前所有频繁项集

In [4]:
# 频繁项集保存到dict中, key为项集长度, value为频繁项集
frequent_itemsets = {}
min_support = 50  # 最小支持度, 用以生成只包含自己的频繁项集
frequent_itemsets[1] = dict((frozenset((movie_id,)), row["Favorable"]) #  这里frozenset的写法一定要这样, 不然会提示TypeError: int not iterable
                            for movie_id, row in num_favorable_by_movie.iterrows()
                            if row["Favorable"]>min_support)
from collections import defaultdict
#  声明一个函数来实现超集和筛选的过程
def find_frequent_itemsets(favorable_review_by_users, k_1_itemsets, min_support):
    counts = defaultdict(int)
    for user, reviews in favorable_review_by_users.items():
        for itemset in k_1_itemsets:
            if itemset.issubset(reviews):
                for other_reviewed_movie in reviews - itemset:
                    current_superset = itemset | frozenset((other_reviewed_movie,))
                    counts[current_superset] += 1
    return dict([(itemset, frequency) for itemset, frequency in counts.items() if frequency>=min_support])
#  遍历k频繁项集长度, 由少到多
#  计算时间
import time
import sys
t1 = time.time()
for k in range(2,5):
    cur_frequent_itemsets= find_frequent_itemsets(favorable_review_by_users, frequent_itemsets[k-1], min_support)
    frequent_itemsets[k] = cur_frequent_itemsets
    if len(cur_frequent_itemsets)==0:
        print("not found frequent itemset for length {0}".format(k))
        sys.stdout.flush()
        break
    else:
        print("found {0} frequent itemsets of length {1}".format(len(cur_frequent_itemsets), k))
        sys.stdout.flush()
del frequent_itemsets[1]
t2 = time.time()
print("time cost:{0}".format(t2-t1))
   

found 268 frequent itemsets of length 2
found 1447 frequent itemsets of length 3
found 4938 frequent itemsets of length 4
time cost:10.617615699768066


## 抽取关联规则 

In [7]:
#  将规则描述为, 如果用户喜欢了一部分电影, 那么会喜欢结论中的电影
candidate_rules = []
for itemset_length, itemset_count in frequent_itemsets.items():
    for itemset in itemset_count.keys():
        for conclusions in itemset:
            premise = itemset - set((conclusions,))
            candidate_rules.append((premise, conclusions))
print(candidate_rules[:5])
# 计算规则的置信度
valid = defaultdict(int)
invalid = defaultdict(int)
for user, reviews in favorable_review_by_users.items():
    for candidate_rule in candidate_rules:
        premise, conclusion = candidate_rule
        if premise.issubset(reviews):
            if conclusion in reviews:
                valid[candidate_rule] += 1
            else:
                invalid[candidate_rule] += 1
#  将数值转换为比例
rule_confidence = {rule: valid[rule] / float(valid[rule] + invalid[rule]) for rule in candidate_rules}
sorted_confidence = sorted(rule_confidence.items(), key=lambda x:x[1], reverse=True)
sorted_confidence[:5]

[(frozenset({260}), 1), (frozenset({1}), 260), (frozenset({1}), 1097), (frozenset({1097}), 1), (frozenset({2762}), 1)]


[((frozenset({589, 1198}), 1196), 1.0),
 ((frozenset({1198, 2571}), 1196), 1.0),
 ((frozenset({1097, 1210}), 1196), 1.0),
 ((frozenset({110, 593, 3578}), 2571), 1.0),
 ((frozenset({110, 1198, 2571}), 1196), 1.0)]

## 评估 

类似抽取关联规则中的置信度评估方法

# 总结 

 - Aprior算法是一类不断尝试扩展频繁项集长度以搜索满足最小支持度的最长频繁项集的算法
 - 该算法在数据集特别大的时候, 搜索空间也会呈爆炸式增长, 当k增大, 组合数C(n,k)会阶乘式的增大, 搜索时间和构建字典占的内存都会爆炸, 所以无法搜索很长的频繁项集
 - 对于集合, 使用set()和frozenset()是不错的方法, 集合类本身的issubset()和issuperset()方法
 - 优雅的构建列表和字典
 - 字典的items()方法要记牢和熟练使用
 - 在长循环中, 使用sys.stdout.flush()可以刷新缓存区, 输出内容, 但是该方法会产生时间消耗(类似产生中断)